### Google drive mount in Google collab Enterprise


In [ ]:
'''
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"
'''

### Install necessary libraries

In [ ]:
#%%capture
!pip install bitsandbytes
!pip install transformers==4.30
!pip install -U peft
!pip install accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install wandb

### Imports

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

### Token for HuggingFace + wandb
It is recommended to use secrets in Google collab or Kaggle

In [ ]:
from google.colab import userdata
HUGGING_FACE_HUB_TOKEN=""
wandb=""
HUGGING_FACE_HUB_TOKEN=userdata.get('HUGGING_FACE_HUB_TOKEN')
secret_wandb=userdata.get('wandb')

In [ ]:
!huggingface-cli login --token $secret_hf

### Model initialization

In [ ]:
base_model = "mistralai/Mistral-7B-v0.1"
new_model = "mistral_7b_NER_fineTuned"

### dataset Preparation

In [ ]:
INS = """<s>[INST]Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.
"""

In [ ]:

import pandas as pd
df = pd.read_csv("/content/df_final_llm (1) (1) copy.csv")
# Rename the column
#df.rename(columns={'Text': 'text'}, inplace=True)
df.head()

,email,entities
0,fm door Quinto Di Treviso to HKG / Totali Quan...,"{'pickup_location': ""Quinto Di Treviso Viale d..."
1,", mi quotereste quanto segue per Favore? , Ori...","{'pickup_location': 'Colico LC', 'quantity': '..."
2,"PORT JEBEL ALI Buon pomeriggio, , potreste quo...","{'delivery_port': 'JEBEL ALI Jebel Ali', 'pick..."
3,"d'oglio /cat lai Buon giorno a tutti , Da Quin...","{'pickup_location': ""d'oglio Quinzano d'Oglio""..."
4,CUCINE INOX // LCL ex. Italy to HK .Atelier Gr...,"{'pickup_state': 'Italy Italy Italy', 'deliver..."


In [ ]:
for i in range(len(df)):
  email = df['email'][i]
  output = df['entities'][i]
  prompt = f"""{INS} ### Input data:{email} ### Output:{output}"""
  df['email'].iloc[i] = prompt
df.drop(columns=['entities'], inplace=True)
df.head()

,email
0,<s>[INST]Your task is to extract the informati...
1,<s>[INST]Your task is to extract the informati...
2,<s>[INST]Your task is to extract the informati...
3,<s>[INST]Your task is to extract the informati...
4,<s>[INST]Your task is to extract the informati...


In [ ]:
df.rename(columns={'email': 'text'}, inplace=True)
df.to_csv('data.csv')

In [ ]:
from datasets import Dataset
#Importing the dataset
dataset = Dataset.from_pandas(df)
dataset

# Modelling

In [ ]:
from accelerate import Accelerator

# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map={"": 0},
        offload_folder="offload/",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="NER/",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)


In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/9 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


### Training

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=3, training_loss=1.4447884559631348, metrics={'train_runtime': 146.5903, 'train_samples_per_second': 0.061, 'train_steps_per_second': 0.02, 'total_flos': 398293481816064.0, 'train_loss': 1.4447884559631348, 'epoch': 1.0})

### Saving the adaopter layer

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
#wandb.finish()
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): 

###. Push to Hub

In [ ]:

try:
    trainer.model.push_to_hub(new_model, use_temp_dir=False)
except:
    print("An exception occurred")

### Initial Results from Adopter layer

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = """
<s>[INST]Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
Buongiorno, ; Ci servirebbe cortesemente Vs quotazione per: ; PESO:  7140 ; COLLI:  7 Pallet ; MISURE: 100X120X130H (6 PLTS)   100X120X160H (1 PLT) ; La dogana verrà fatta a stabilimento, pertanto si richiede solo lo scarico bolla. ; La attendiamo ; Grazie mille ; Nicolo Esibiti ; Export Sales Department ; T +39 010 2717399 ; Casasco & Nardi S.p.A. ; Palazzo Andrea Pitto ; Piazza Fossatello, 3 ; 16124, Genova ; www.casasconardi.com ;

### Output:

"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=32000)
result = pipe(f"<s>[INST] {prompt} [/INST]</s>")
print(result[0]['generated_text'])

<s>[INST] 
<s>[INST]Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate t